# Container - Building an Application


In this notebook you will learn how to use \<container> resources to represent structured data for an application.

In this notebook we will continue to use the resource structure created in the previous notebooks of this lecture.


- Create the application's data structure in the CSE
- Fill the structure with simulated data
- Retrieve and visualize the data


## Intitialization

<div class="alert alert-block alert-info" style="border-radius: 10px;">
The following section imports and initializes necessary functions and configurations. 
It must be executed once for each notebook before running any of cells.</div>

In [ ]:
%run ../src/init.py

## Create the \<container> structure

Alice bought a connected sensor device that can measure temperature and humidity. She likes to store the data from that device in the CSE. For that she will create a small hierarchie of \<container> resources that will represent her new device.

This figure shows the intended structure for the sensor device:

![](containerSensor.png)

The following three requests will setup this structure.


anschliessend CIN mit random numbers

### Create \<container> resource "aliceSensor"

In [ ]:
CREATE (                                            # CREATE request
    target                  = f'{cseRN}/aeAlice',   # Create a <cnt> under the <AE>
    originator              = 'CAlice',             # Request's originator (mandatory)
    requestIdentifier       = '123',                # Request identifier (mandatory)
    releaseVersionIndicator = '3',                  # Request's release version indicator (mandatory)
    resourceType            = 3,                    # The request creates an <cnt> resource
    
    # Request Content
    content = {
        "m2m:cnt": {
            "rn"  : "aliceSensor"                   # Give the <cnt> resource the name 'aliceSensor'
        }
    }
)

### Create \<container> resource "temperature"

In [ ]:
CREATE (                                                        # CREATE request
    target                  = f'{cseRN}/aeAlice/aliceSensor',   # Create a <cnt> under "aliceSensor"
    originator              = 'CAlice',                         # Request's originator (mandatory)
    requestIdentifier       = '123',                            # Request identifier (mandatory)
    releaseVersionIndicator = '3',                              # Request's release version indicator (mandatory)
    resourceType            = 3,                                # The request creates an <cnt> resource
    
    # Request Content
    content = {
        "m2m:cnt": {
            "mni" : 10,                                         # Limit the number of <cin> to 10
            "rn"  : "temperature"                               # Give the <cnt> resource the name 'temperature'
        }
    }
)

### Create \<container> resource "humidity"

In [ ]:
CREATE (                                                        # CREATE request
    target                  = f'{cseRN}/aeAlice/aliceSensor',   # Create a <cnt> under "aliceSensor"
    originator              = 'CAlice',                         # Request's originator (mandatory)
    requestIdentifier       = '123',                            # Request identifier (mandatory)
    releaseVersionIndicator = '3',                              # Request's release version indicator (mandatory)
    resourceType            = 3,                                # The request creates an <cnt> resource
    
    # Request Content
    content = {
        "m2m:cnt": {
            "mni" : 10,                                         # Limit the number of <cin> to 10
            "rn"  : "humidity"                                  # Give the <cnt> resource the name 'humidity'
        }
    }
)

After this request the structure should be complete. 

## Add sensor data to the \<container> resources

Now that the structure is complete Alice can connect her new device and fill the structure with sensor data.


<div class="alert alert-block alert-info" style="border-radius: 10px;">
We will simulate the device data by a couple of random numbers (using the <b><tt>randomTemperature()</tt></b> and <b><tt>randomHumidity()</tt></b> functions). Those are alll generated at once by repeatingly sending a CREATE request.
    
These requests produce a bit more output.
</div>

### Create temperature data

In [ ]:
repeat(
    times   = 4,
    
    request = lambda : CREATE (                                                 # CREATE request
        target                  = f'{cseRN}/aeAlice/aliceSensor/temperature',   # Create a <cin> under the temperature <cnt>
        originator              = 'CAlice',                                     # Request's originator (mandatory)
        requestIdentifier       = '123',                                        # Request identifier (mandatory)
        releaseVersionIndicator = '3',                                          # Request's release version indicator (mandatory)
        resourceType            = 4,                                            # The request creates a <cin> resource
    
        # Request Content
        content = {
            "m2m:cin": {
                "con": randomTemperature()                                      # Produce "temperature" data
            }
        }
    )
)

### Create humidity data

In [ ]:
repeat(
    times   = 4,
    
    request = lambda : CREATE (                                                 # CREATE request
        target                  = f'{cseRN}/aeAlice/aliceSensor/humidity',      # Create a <cin> under the temperature <cnt>
        originator              = 'CAlice',                                     # Request's originator (mandatory)
        requestIdentifier       = '123',                                        # Request identifier (mandatory)
        releaseVersionIndicator = '3',                                          # Request's release version indicator (mandatory)
        resourceType            = 4,                                            # The request creates a <cin> resource
    
        # Request Content
        content = {
            "m2m:cin": {
                "con": randomHumidity()                                         # Produce "temperature" data
            }
        }
    )
)

## Retrieve and present the data 

Alice now has a nicely filled data structure. She can execute the requests a couple of times, but the internal management will make sure that the number of \<contentInstance> resource will always be limited to the specified maximum number (10).

The next step for Alice is to write an application that presents the data nicely.

<div class="alert alert-block alert-success" style="border-radius: 10px">
This exercise will retrieve the available data from the &lt;contentInstance> resources with so-called <i>conditional retrieve</i> requests. With these we can query and retrieve one or multiple resources from the resource tree. This variant of oneM2M RETRIEVE requests will be explained in detail in a separate lecture, but it is very convenient to use them here already.
</div>

<div class="alert alert-block alert-info" style="border-radius: 10px;">
In this exercise we not only will send requests to the CSE but also will do some small calculations and makes use of Python's <i>pandas</i> and <i>matplotlib</i> libraries for presenting the data.
    
You may notice the <b><tt>_verbose</tt></b> argument in the requests below. This is a non-oneM2M parameter that we use in our exercises to disable verbose printing of all the request and response details in order to concentrate on other parts of code and output.
    
Another note: the <b><tt>response()</tt></b> function used in the code below returns the last request's response content as a dictionary, which we then can conveniently process further.
</div>



In [ ]:
# Retrieve the temperature values
RETRIEVE (                                                                 # RETRIEVE request
    target                  = f'{cseRN}/aeAlice/aliceSensor/temperature',  # Target resource is the <cin> under Alice's <cnt>
    originator              = 'CAlice',                                    # Request's originator is CAlice (mandatory)
    requestIdentifier       = '123',                                       # Request identifier (mandatory)
    releaseVersionIndicator = '3',                                         # Request's release version indicator (mandatory)
    
    # Parameterize the conditional retrieve
    filterUsage             = 2,                                           # Conditional retrieve
    resourceType            = 4,                                           # Only retrieve <contentInstance> resources
    resultContent           = 8,                                           # Only retrieve the child resources
    
    _verbose                = False                                        # Internall: Don't print the request and response
)

# Extract the content values from the response and put them into an array
temperatures = [ int(cin['con']) for cin in findXPath(response(), 'm2m:cnt/m2m:cin') ]


# Retrieve the humidity values
RETRIEVE (                                                                 # RETRIEVE request
    target                  = f'{cseRN}/aeAlice/aliceSensor/humidity',     # Target resource is the <cin> under Alice's <cnt>
    originator              = 'CAlice',                                    # Request's originator is CAlice (mandatory)
    requestIdentifier       = '123',                                       # Request identifier (mandatory)
    releaseVersionIndicator = '3',                                         # Request's release version indicator (mandatory)
    
    # Parameterize the conditional retrieve
    filterUsage             = 2,                                           # Conditional retrieve
    resourceType            = 4,                                           # Only retrieve <contentInstance> resources
    resultContent           = 8,                                           # Only retrieve the child resources
    
    _verbose                = False                                        # Internal: Don't print the request and response
)

# Extract the content values from the response and put them into an array
humidities = [ int(cin['con']) for cin in findXPath(response(), 'm2m:cnt/m2m:cin') ]


# Make both arrays the same length in case we have different numbers of measurements.
# This is necessary for the plot below
ml = max(len(temperatures), len(humidities))
temperatures += [0] * (ml - len(temperatures))
humidities   += [0] * (ml - len(humidities))


# Create nice bar charts from our values
import pandas as pd

source = pd.DataFrame({
    'Measurement' : list(range(1,ml+1)),
    'Temperature' : temperatures,
    'Humidity'    : humidities,

})
source.plot.bar(y=['Temperature', 'Humidity'], x='Measurement', subplots=True, layout=(1,2), figsize=(15,3));


&nbsp;